# 12. HDF5 to Video

*Made by David Serrano for his Final Bachelor Thesis*

This file transforms the hdf5 file of a sequence into a .avi sequence to be able to display the stream

## 12.1 Environment Preparation

In [ ]:
# Useful general imports
from google.colab import drive

In [ ]:
# mount the google drive folder to access to all the files in there
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Add the folder called utils that has all the custom modules in .py files
import sys
sys.path.append('/content/drive/MyDrive/TFG/utils')

In [ ]:
from paths import PROJECT_ROOT, HDF5_FOLDER, JSON_FOLDER, USABLE_SEQUENCES, MAX_SQUARE_LEN

## 12.2. 

In [ ]:
EXAMPLE_FILE = '20181002.047_AEQ31'

In [ ]:
# Load the sequnce example
import h5py
import time
from image_tools import computeDarkImage

print('Loading sequence file...')
f = h5py.File(HDF5_FOLDER + EXAMPLE_FILE + '.h5', 'r', driver='core')
t = time.time()
images = f['ROIP']['ROIP1']['ROIP1Data'][()]
timestamps = f['ROIP']['ROIP1']['ROIP1W7XTime'][()]
elapsed = time.time() - t
f.close()
print('Elapsed time reading: {}'.format(elapsed))

darkIm = computeDarkImage(images)

Loading sequence file...
Elapsed time reading: 65.58880925178528
Computing Dark image...
Elapsed time computing dark image: 3.5400586128234863


In [ ]:
from hdf5Loader import hdf5Loader
from image_tools import preProcess
import cv2
import numpy as np

loader = hdf5Loader(HDF5_FOLDER + EXAMPLE_FILE + '.h5')
images = []

for index in range(loader.nFrames()):
    image = loader.loadImage(index) - darkIm
    image = preProcess(image)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    image = cv2.putText(image, str(index),
                            (50, image.shape[0]-50), cv2.FONT_HERSHEY_SIMPLEX,
                            2, (0,0,255), 2, cv2.LINE_AA, False)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    images.append(image)


images = np.array(images)

In [ ]:
from PIL import Image

for index in range(len(images)):
    im = Image.fromarray(images[index])
    im.save(PROJECT_ROOT + '/data/temp/img' + str(index) + '.png')

In [ ]:
import cv2
import os

video_name = PROJECT_ROOT + '/data/' + EXAMPLE_FILE + '.avi'
folder = PROJECT_ROOT + '/data/temp/'

images = [img for img in os.listdir(folder) if img.endswith('.png')]
frame = cv2.imread(os.path.join(folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MJPG'), 100, (width, height))

for image in images:
    video.write(cv2.imread(os.path.join(folder, image)))

video.release()